In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler

# ✅ Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [21]:
# Cell 2: Load and Prepare Data
df = pd.read_csv("IL_T800_cleaned1.csv")

# Check and drop NaN values
df.dropna(inplace=True)

# Separate features (X) and label (y)
X = df.drop(columns=['label']).values
y = df['label'].values

# Balance the dataset with RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)
print(f"Balanced dataset: {np.bincount(y_resampled)}")

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_resampled)


Balanced dataset: [10000 10000]


In [22]:
# Cell 3: Create Sequences for LSTM
timesteps = 30  # Adjusted sequence length

def create_sequences(data, labels, seq_len=30):
    seqs, labs = [], []
    for i in range(len(data) - seq_len + 1):
        seqs.append(data[i : i + seq_len])
        labs.append(labels[i + seq_len - 1])
    return np.array(seqs), np.array(labs)

X_seq, y_seq = create_sequences(X_scaled, y_resampled, seq_len=timesteps)

print("Sequence X_seq shape:", X_seq.shape)
print("Sequence y_seq shape:", y_seq.shape)


Sequence X_seq shape: (19971, 30, 19)
Sequence y_seq shape: (19971,)


In [23]:
# Cell 4: TimeSeriesSplit for Train-Test Split
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X_seq):
    X_train, X_test = X_seq[train_index], X_seq[test_index]
    y_train, y_test = y_seq[train_index], y_seq[test_index]

print("X_train shape:", X_train.shape, "X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape, "y_test shape:", y_test.shape)


X_train shape: (16643, 30, 19) X_test shape: (3328, 30, 19)
y_train shape: (16643,) y_test shape: (3328,)


In [24]:
# Cell 5: Prepare Tensors and DataLoader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test,  dtype=torch.long).to(device)

batch_size = 32  # Reduced batch size for better gradient updates
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor,  y_test_tensor)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [25]:
# Cell 6: Define LSTM Model
class DeeperLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, num_layers=2, dropout=0.5):
        super(DeeperLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim // 2, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_step = lstm_out[:, -1, :]
        x = self.fc1(last_step)
        x = self.relu(x)
        out = self.fc2(x)
        return out

num_features = X.shape[1]
model = DeeperLSTM(input_dim=num_features).to(device)


In [26]:
# Cell 7: Define Weighted Loss and Optimizer
class_weights = torch.tensor([0.5, 1.5]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)  # Added L2 regularization

print("Criterion:", criterion)
print("Optimizer:", optimizer)


Criterion: CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.0001
)


In [27]:
# Cell 8: Train the Model
epochs = 100
for epoch in range(1, epochs + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for Xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += yb.size(0)
        correct += (predicted == yb).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc  = 100.0 * correct / total
    print(f"Epoch {epoch}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")


Epoch 1/100 | Loss: 0.4108 | Acc: 68.14%
Epoch 2/100 | Loss: 0.0954 | Acc: 97.00%
Epoch 3/100 | Loss: 0.0627 | Acc: 98.46%
Epoch 4/100 | Loss: 0.0569 | Acc: 98.70%
Epoch 5/100 | Loss: 0.0536 | Acc: 98.89%
Epoch 6/100 | Loss: 0.0514 | Acc: 98.72%
Epoch 7/100 | Loss: 0.0418 | Acc: 99.05%
Epoch 8/100 | Loss: 0.0422 | Acc: 99.14%
Epoch 9/100 | Loss: 0.0397 | Acc: 99.18%
Epoch 10/100 | Loss: 0.0405 | Acc: 99.12%
Epoch 11/100 | Loss: 0.0355 | Acc: 99.30%
Epoch 12/100 | Loss: 0.0325 | Acc: 99.40%
Epoch 13/100 | Loss: 0.0381 | Acc: 99.21%
Epoch 14/100 | Loss: 0.0318 | Acc: 99.47%
Epoch 15/100 | Loss: 0.0331 | Acc: 99.41%
Epoch 16/100 | Loss: 0.0319 | Acc: 99.38%
Epoch 17/100 | Loss: 0.0356 | Acc: 99.50%
Epoch 18/100 | Loss: 0.0315 | Acc: 99.48%
Epoch 19/100 | Loss: 0.0314 | Acc: 99.25%
Epoch 20/100 | Loss: 0.0280 | Acc: 99.49%
Epoch 21/100 | Loss: 0.0241 | Acc: 99.58%
Epoch 22/100 | Loss: 0.0293 | Acc: 99.39%
Epoch 23/100 | Loss: 0.0301 | Acc: 99.44%
Epoch 24/100 | Loss: 0.0295 | Acc: 99.47%
E

In [28]:
# Cell 9: Evaluate the Model
model.eval()
all_preds, all_true = [], []
with torch.no_grad():
    for Xb, yb in test_loader:
        logits = model(Xb)
        _, predicted = torch.max(logits, 1)
        all_preds.append(predicted.cpu().numpy())
        all_true.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_true = np.concatenate(all_true)

acc  = accuracy_score(all_true, all_preds) * 100
prec = precision_score(all_true, all_preds, pos_label=1, zero_division=0) * 100
rec  = recall_score(all_true, all_preds, pos_label=1, zero_division=0) * 100
f1   = f1_score(all_true, all_preds, pos_label=1, zero_division=0) * 100

print("\n✅ Final Test Results:")
print(f"Accuracy:   {acc:.2f}%")
print(f"Precision:  {prec:.2f}%")
print(f"Recall:     {rec:.2f}%")
print(f"F1-score:   {f1:.2f}%")



✅ Final Test Results:
Accuracy:   99.79%
Precision:  100.00%
Recall:     99.79%
F1-score:   99.89%


In [19]:
epochs = 100
for epoch in range(1, epochs + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for Xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += yb.size(0)
        correct += (predicted == yb).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc  = 100.0 * correct / total
    scheduler.step(epoch_loss)
    print(f"Epoch {epoch}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")


Epoch 1/100 | Loss: 0.0422 | Acc: 60.08%
Epoch 2/100 | Loss: 0.0422 | Acc: 60.08%
Epoch 3/100 | Loss: 0.0421 | Acc: 60.08%
Epoch 4/100 | Loss: 0.0414 | Acc: 61.08%
Epoch 5/100 | Loss: 0.0342 | Acc: 73.00%
Epoch 6/100 | Loss: 0.0325 | Acc: 74.84%
Epoch 7/100 | Loss: 0.0305 | Acc: 77.16%
Epoch 8/100 | Loss: 0.0294 | Acc: 78.42%
Epoch 9/100 | Loss: 0.0286 | Acc: 79.26%
Epoch 10/100 | Loss: 0.0275 | Acc: 80.45%
Epoch 11/100 | Loss: 0.0269 | Acc: 81.24%
Epoch 12/100 | Loss: 0.0267 | Acc: 81.29%
Epoch 13/100 | Loss: 0.0266 | Acc: 81.51%
Epoch 14/100 | Loss: 0.0257 | Acc: 82.14%
Epoch 15/100 | Loss: 0.0258 | Acc: 82.18%
Epoch 16/100 | Loss: 0.0250 | Acc: 83.22%
Epoch 17/100 | Loss: 0.0245 | Acc: 83.40%
Epoch 18/100 | Loss: 0.0239 | Acc: 83.81%
Epoch 19/100 | Loss: 0.0235 | Acc: 84.10%
Epoch 20/100 | Loss: 0.0236 | Acc: 84.12%
Epoch 21/100 | Loss: 0.0233 | Acc: 84.32%
Epoch 22/100 | Loss: 0.0228 | Acc: 84.52%
Epoch 23/100 | Loss: 0.0227 | Acc: 84.48%
Epoch 24/100 | Loss: 0.0234 | Acc: 83.95%
E

In [20]:
model.eval()
all_preds, all_true = [], []
with torch.no_grad():
    for Xb, yb in test_loader:
        logits = model(Xb)
        _, predicted = torch.max(logits, 1)
        all_preds.append(predicted.cpu().numpy())
        all_true.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_true = np.concatenate(all_true)

acc  = accuracy_score(all_true, all_preds) * 100
prec = precision_score(all_true, all_preds, pos_label=1) * 100
rec  = recall_score(all_true, all_preds, pos_label=1) * 100
f1   = f1_score(all_true, all_preds, pos_label=1) * 100

print("\n✅ Final Test Results:")
print(f"Accuracy:   {acc:.2f}%")
print(f"Precision:  {prec:.2f}%")
print(f"Recall:     {rec:.2f}%")
print(f"F1-score:   {f1:.2f}%")



✅ Final Test Results:
Accuracy:   83.04%
Precision:  100.00%
Recall:     83.04%
F1-score:   90.74%
